In [31]:
import random
import copy
import heapq
import numpy as np
import matplotlib.pyplot as plt

from time import time
from typing import Set

# PARAMETERS
 Update the parameters

In [45]:
PAUSE_TIME = 0.1
goal_positions: dict = {}
heuristic: str = "manhattan"
# Number of random moves for generating the puzzle
randomize: int = 200
algorithm: str = "astar"
#algorithm: str = "greedy"

# NODE class

In [33]:
global goal, goal_positions
class Node:
    """
    Class representing a node in the puzzle
    """
    def __init__(self, tiles: list[int] = None, parent = None, g = 0, h = 0, f = 0):
        """
        Initialize the node
        Args: tiles(List[int], optional): The tiles of the puzzle
        parent (Node, optional): The parent of the Node
        g (int, optional) : The cost to reach this node
        h (int, optional) : The heuristic cost estimate of the cheapest path from this node to the goal
        f (int, optional) : The estimated cost of the cheapest solution through this node
        """
        
        if tiles is None:
            tiles = list()
        
        self.tiles = tiles
        self.parent = parent
        self.g = g
        self.h = h
        self.f = f
        
    def is_goal(self):
        """
        Check if this node is the goal
        Return: bool, True if this is the goal, False otherwise
        """
        return self.tiles == goal.tiles
        
    def get_empty(self):
        """
        Get the position of the empty tile
        Returns: tuple, The position of the empty tile
        """
            
        tiles = self.tiles
        for i in range(size * size):
            if tiles[i] == 0:
                return i // size, i % size
        return 0, 0
        
    def manhattan_distance(self):
        """
        Calculate the Manhattan distance from the current state to the goal state.
        This is the sum of the distances (in the horizontal and vertical directions)
        from each tile to its goal position. It represents the minimum number of moves
        that each tile must make to reach its goal position.
        """
        total_sum = 0
        tiles = self.tiles
            
        for i in range(size * size):
            if tiles[i] == 0:
                continue
            else:
                x, y = goal_positions[tiles[i]]
                total_sum += abs(x - i // size) + abs(y - i % size)
                
            # update the cost
            self.h = total_sum
            self.g = self.parent.g + 1
            self.f = self.h
            if algorithm != 'greedy':
                self.f += self.g
        
    def euclidean_distance(self):
        """
        Calculate the Euclidean distance from the current state to the goal state.
        The Euclidean distance is also knowed as "as the crow flies" distance.
        """
        total_sum = 0
        tiles = self.tiles
            
        for i in range(size * size):
            if tiles[i] == 0:
                continue
            else:
                goal_x, goal_y = goal_positions[tiles[i]]
                current_x, current_y = i // size, i % size
                # Euclidian distance between the goal position and the current position
                total_sum += ( (goal_x - current_x) ** 2 + (goal_y - current_y) ** 2 ) ** 0.5
            
        # Update the cost
        self.h = total_sum
        self.g = self.parent.g + 1
        self.f = self.h
        if algorithm != "greedy":
            self.f += self.g
        
    def hamming_distance(self):
        """
        Calculate the Hamming distance from the current state to the goal state.
        Misplaced Tiles (Hamming distance): This is the toal number of tiles 
        that are not in their correct place
        """
        total_sum = 0
        tiles = self.tiles
            
        for i in range(size * size):
            if tiles[i] == 0:
                continue
            elif tiles[i] != goal.tiles[i]:
                total_sum += 1
            
        # Update the cost
        self.h = total_sum
        self.g = self.parent.g + 1
        self.f = self.h
        if algorithm != "greedy":
            self.f += self.g
        
    def linear_conflict_manhattan(self):
        """
        Calculate the linear conflict from the current state to the goal state.
        This involves counting the number of pairs of tiles that are in the wrong
        order compared to the goal. Use Manhattan, then add 2 for each pair of
        tiles that are in the same row or column and must be swapped
        """
            
        self.manhattan_distance()
        total_sum = self.h
        tiles = self.tiles
            
        for row in range(size):
            row_tiles = [tiles[i] for i in range((row * size), (row + 1) * size) if tiles[i] != 0]
            for i in range(len(row_tiles)):
                for j in range(i + 1, len(row_tiles)):
                    if ((goal_positions[row_tiles[i]][0] == goal_positions[row_tiles[j]][0]) and (goal_positions[row_tiles[i]][1] > goal_positions[row_tiles[j]][1])):
                        total_sum += 2
            
        for col in range(size):
            col_tiles = [tiles[i * size + col] for i in range(size) if tiles[i * size + col] != 0]
            for i in range(len(col_tiles)):
                for j in range(i + 1, len(col_tiles)):
                    if ((goal_positions[col_tiles[i]][1]== goal_positions[col_tiles[j]][1]) and (goal_positions[col_tiles[i]][0] > goal_positions[col_tiles[j]][0])):
                        total_sum += 2
            
        # Update the cost
        self.h = total_sum
        self.g = self.parent.g + 1
        self.f = self.h
        if algorithm != "greedy":
                self.f += self.g
        
    def __eq__(self, other):
        """
        Check if this node is equal to another node
        other (Node): The other Node
        Return: bool, True if they are equal, False otherwise
        """
            
        if other is None:
            return False
        return self.tiles == other.tiles and self.f == other.f
        
    def __lt__(self, other):
        """
        Check if this node is less than another node.
        other (Node): The other Node.
        Returns: bool, True if this node is less than the other node, False otherwise
        """
        return self.f < other.f
            
    def __hash__(self):
        """
        Calculate the has of this node
        Return: int, the has of this node
        """
            
        return hash(tuple(self.tiles))
        
    def get_children(self):
        """
        Generate the childer of this node.
        Returns: List, the children of this node
        """
            
        tiles = self.tiles
        # Get the position of the 0
        x, y = self.get_empty()
        new_tiles = []
            
        if (x + 1) < size:
            new = copy.deepcopy(tiles)
            # Swap the near element in the 0 cell
            new[x * size + y] = new[(x + 1) * size + y]
            new[(x + 1) * size + y] = 0
            new_tiles.append(new)
        if (x - 1) > -1:
            new = copy.deepcopy(tiles)
            new[x * size + y] = new[(x - 1) * size + y]
            new[(x - 1) * size + y] = 0
            new_tiles.append(new)
        if (y + 1) < size:
            new = copy.deepcopy(tiles)
            new[x * size + y] = new[x * size + y + 1]
            new[x * size + y + 1] = 0
            new_tiles.append(new)
        if (y - 1 ) > -1:
            new = copy.deepcopy(tiles)
            new[x * size + y] = new[x * size + y - 1]
            new[x * size + y - 1] = 0
            new_tiles.append(new)
           
        ret = []
        for i in new_tiles:
            child = Node(i, self)
            # Apply the Heuristic strategy
            if heuristic == "manhattan":
                child.manhattan_distance()
            elif heuristic == "euclidean":
                child.euclidean_distance()
            elif heuristic == "hamming":
                child.hamming_distance()
            elif heuristic == "lconflict":
                child.linear_conflict_manhattan()
                
            if self.parent is not None and child.tiles == self.parent.tiles:
                continue
                
            ret.append(child)
            
        return ret

# Priority Queue Class

In [34]:
class PriorityQueue:
    """
    Class representing a priority Queue
    """
    def __init__(self):
        """
        Initialize the priority Queue
        """
        self.elements = []
        
    def __len__(self):
        """
        Get the number of elements in the priority queue.
        Returns: int, the number of elements
        """
        return len(self.elements)

    def __getitem__(self, item):
        """
        Get the item at the given index.
        Args: item (int), The index of the item
        Returns: Any, the item at the given index
        """
        return self.elements[item]

    def empty(self):
        """
        Check if the priority queue is empty
        Returns: bool, True if the priority queue is empty, False otherwise
        """
        return len(self.elements) == 0
    
    def put(self, node: Node):
        """
        Add a node to the priority queue.
        Args: node (Node), the node to add.
        """
        heapq.heappush(self.elements, node)
    
    def get(self):
        """
        Remove and return the smallest node from the priority queue.
        Returns: Node, the smallest node
        """
        return heapq.heappop(self.elements)

# Functions for generation and printing

In [35]:
def generate_random_puzzle(puzzle_size: int) -> Node:
    """
    Generates a random valid puzzle of the specified size.
    
    This function takes the size of the puzzle as input and generates a random valid puzzle
    of that size. The puzzle is generated by starting with an ordered spiral puzzle and then
    performing a random sequence of valid moves, moving the zero tile with an adjacent tile
    
    Returns: Node, the initial state of the generated puzzle
    """
    
    tiles = generate_ordered_spiral_puzzle(puzzle_size)
    initial_state = Node(tiles)
    #print(initial_state.tiles)
    
    for _ in range(randomize):
        empty_x, empty_y = initial_state.get_empty()
        #print("start",empty_x, empty_y)
        possible_moves = []
        # Define all possible moves
        if empty_x > 0:
            possible_moves.append((empty_x - 1, empty_y))
        if empty_x < puzzle_size - 1:
            possible_moves.append((empty_x + 1, empty_y))
        if empty_y > 0:
            possible_moves.append((empty_x, empty_y - 1))
        if empty_y < puzzle_size - 1:
            possible_moves.append((empty_x, empty_y + 1))
        
        # Select a random choice
        move_x, move_y = random.choice(possible_moves)
        #print("move",move_x, move_y)
        # From matrix coords to vector index
        empty_pos = empty_x * puzzle_size + empty_y
        move_pos = move_x * puzzle_size + move_y
        
        # Swap the two cells
        initial_state.tiles[empty_pos], initial_state.tiles[move_pos] = initial_state.tiles[move_pos], initial_state.tiles[empty_pos]
    
    return initial_state


def generate_ordered_spiral_puzzle(puzzle_size: int) -> list:
    """
    Generates an ordered spiral puzzle of the specified size.
    This function takes the size of the puzzle as input and generates an ordered spiral puzzle
    of that size. The puzzle is generated by starting with an empty grid and filling it in a spiral pattern with the tile values.
    
    Return: list, the ordered spiral puzzle tiles
    """
    
    tiles = [[0] * puzzle_size for _ in range(puzzle_size)]
    # Define the possible actions
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    direction_idx = 0
    current_x, current_y = 0, 0
    count = 1
    
    available_numbers = list(range(1, puzzle_size * puzzle_size + 1))
    
    while count < puzzle_size * puzzle_size:
        tiles[current_x][current_y] = available_numbers[count - 1]
        count += 1
        
        next_x = current_x + directions[direction_idx][0]
        next_y = current_y + directions[direction_idx][1]
        
        # Check if the move is possible
        if (
            next_x < 0
            or next_x >= puzzle_size
            or next_y >= puzzle_size
            or tiles[next_x][next_y] != 0
        ):
            direction_idx = (direction_idx + 1) % 4
        
        current_x += directions[direction_idx][0]
        current_y += directions[direction_idx][1]
    
    tiles = [tile for row in tiles for tile in row]
    
    return tiles

def generate_goal(size):
    """
    Generates the goal state fot the puzzle.
    Spiral puzzle
    """
    
    global goal, goal_positions
        
    tiles = np.arange(size* size)

    count = 1
    col = 0
    row = 0
    iteration = 0

    while count < size * size:
        while col < size - iteration:
            tiles[row * size + col] = count
            goal_positions[count] = (row, col)
            count += 1
            col += 1
        if count > size * size:
            break
        
        col -= 1
        row += 1
        
        while row < size - iteration:
            tiles[row * size + col] = count
            goal_positions[count] = (row, col)
            count += 1
            row += 1
        if count > size * size:
            break
        
        col -= 1
        row -= 1
        
        while col >= iteration:
            tiles[row * size + col] = count
            goal_positions[count] = (row, col)
            count += 1
            col -= 1
        if count > size * size:
            break
        
        col += 1
        row -= 1
        
        while row > iteration:
            tiles[row * size + col] = count
            goal_positions[count] = (row, col)
            count += 1
            row -= 1
        if count > size * size:
            break
        
        col += 1
        row += 1
        iteration += 1
        
    if size % 2 == 0:
        tiles[size // 2 * size + size // 2 - 1] = 0
        goal_positions[0] =   (size // 2, size // 2 - 1)
    else:
        tiles[size // 2 * size + size // 2] = 0
        goal_positions[0] = (size // 2, size // 2)

    goal.tiles = list(tiles)
    print("Goal state:")
    printNodeText(goal)
    
def printNodeText(node: Node):
    """
    Print the puzzle tile
    """
    tiles = node.tiles
    for i in range(size):
        for j in range(size):
            print(tiles[i * size + j], end = " ")
            
def printNode(node: Node):
    """
    Visualizes the statge of the puzzle using matplotlib.
    The function first clears the current figure (if it exists), then reshapes the
    flat list of tiles into a 2D array to represent the puzzle grid.
    IT then uses matplotlib to create an image of the grid, where each tile's values is
    represented as a different color. The values of each tile is also printed at the center of the corresponding square in the grid.
    After drawing the grid, the function pauses for a set amount of time before returning.
    """
    
    plt.clf()
    tiles = node.tiles
    grid = np.array(tiles).reshape((size, size))
    
    plt.imshow(grid, cmap='viridis')
    for i in range(size):
        for j in range(size):
            plt.text(j, i, grid[i,j], ha="center", va="center", color="black")
    plt.xticks([])
    plt.yticks([])
    plt.title("Solution")
    plt.draw()
    plt.pause(PAUSE_TIME)

# Function for solving the puzzle

In [36]:
def is_explored(node: Node, explored: Set):
    """
    Checks whether the given node is in the set of explored nodes.
    This function takes a node and a set of explored nodes. It returns True if the node is in the set,
    and False otherwhise.
    """
    if node in explored:
        return True
    return False

def findGoal(goal_pos):
    """
    Locates the position of the specified tile in the goal state.
    This function iterates over the tiles in the goal state to find the one matching the specified 'goal_pos'.
    When it finds a match, it calculates and returns the (row, col) position of the tile.
    """
    global goal
    tiles = goal.tiles
    for i in range(size * size):
        if tiles[i] == goal_pos:
            return i // size, i % size

def solve(starting: Node):
    """
    Solves the puzzle using A* search algorithm.
    This function implements the A* search algorithm to find the solution to the puzzle.
    It maintains an open list (priority queue base on f-score) and a closed list (set of explored nodes).
    It strarts with the initial state of the puzzle and explores child nodes by moving the empty tile in all possible directions.
    It continues this process until it finds the goal state of the puzzle.
    Returns: Node, the final state of the puzzle if a solution is found, otherwise None.
    """
    
    open_list = PriorityQueue()
    open_list.put(starting)
    closed_list = set()
    # Number of moves
    cost = 0
    
    while len(open_list) > 0:
        best_node: Node = open_list.get()
        
        if best_node.is_goal():
            #print()
            #print("Open list length: " + str(len(open_list)))
            #print("Closed list length: " + str(len(closed_list)))
            return best_node, cost
        
        if is_explored(best_node, closed_list):
            continue
        
        children = best_node.get_children()
        
        for child in children:
            open_list.put(child)
        
        closed_list.add(best_node)
        cost += 1
    
    return None, cost

In [37]:
def main():
    puzzle = generate_random_puzzle(size)
    print(f"starting puzzle : {puzzle.tiles}")
    generate_goal(size)
    
    start = time()
    solution, cost = solve(puzzle)
    end = time()

    print()
    print(f'time: {round(end-start, 2)}s')
    print(f'Valid solution: {solution.tiles == goal.tiles}')
    print(f'Cost: {cost}')

# MAIN block
## Before running this block execute all the others block
select the size of the puzzle\
run main ==> solve the puzzle

In [46]:
size = 7
goal: Node = Node()


In [47]:
heuristic = "manhattan"
# This code apply the heuristic manhattan
main()

starting puzzle : [1, 2, 3, 4, 5, 6, 7, 24, 25, 26, 42, 27, 29, 8, 23, 41, 28, 43, 9, 10, 30, 22, 40, 39, 48, 44, 0, 11, 21, 38, 17, 47, 34, 32, 33, 20, 37, 16, 31, 46, 12, 13, 19, 18, 15, 36, 45, 35, 14]
Goal state:
1 2 3 4 5 6 7 24 25 26 27 28 29 8 23 40 41 42 43 30 9 22 39 48 0 44 31 10 21 38 47 46 45 32 11 20 37 36 35 34 33 12 19 18 17 16 15 14 13 
time: 24.42s
Valid solution: True
Cost: 283195


In [ ]:
# For a fixed size find the best heuristic strategy!
performance = {"cost": 999999, "algorithm":"", "heuristic": ""}
# hamming is a very slow strategy, remove it if the size is too big
strategies = ["manhattan",  "euclidean", "lconflict", "hamming"]
goal: Node = Node()
size = 7
puzzle = generate_random_puzzle(size)
generate_goal(size)

for tech in strategies:
    heuristic = tech
    puzzle_to_solve = copy.deepcopy(puzzle)
    start = time()
    solution, cost = solve(puzzle_to_solve)
    end = time()

    print()
    print(f'heuristic: {heuristic}')
    print(f'time: {round(end-start, 2)}')
    print(f'Valid solution: {solution.tiles == goal.tiles}')
    print(f'Cost: {cost}')
    
    # Update performance
    if cost < performance["cost"]:
        performance["cost"] = cost
        performance["algorithm"] = algorithm
        performance["heuristic"] = heuristic
        
print()
print(f'Best perfomer for puzzle size {size} is {performance["heuristic"]} whith a cost of {performance["cost"]}')